<a href="https://colab.research.google.com/github/RBBRONDANI/DegiroAPI/blob/master/Degiro_ETF_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Obter lista de ETFs gratuitos Degiro**

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import time

In [ ]:
csv_path = "/content/drive/MyDrive/Arquivos Python/FreeETFList.csv"
free_ETF = pd.read_csv(csv_path)
free_ETF = free_ETF.drop(labels='0', axis=1) 
free_ETF

,ISIN,Nome,Moeda,Bolsa
0,LU1681046931,AMUNDI ETF CAC 40,EUR,Euronext Paris
1,LU1681044480,AMUNDI ETF EM ASIA,EUR,Euronext Paris
2,LU1681047236,AMUNDI ETF ES50 C,EUR,Euronext Paris
3,LU1681045370,AMUNDI ETF MSCI EM,EUR,Euronext Paris
4,LU1681043599,AMUNDI ETF MSCI WR,EUR,Euronext Paris
...,...,...,...,...
195,LU0514695690,Xtrackers MSCI China UCITS ETF 1C,EUR,Xetra
196,IE00BTJRMP35,Xtrackers MSCI Emerging Markets UCITS ETF 1C,EUR,Xetra
197,IE00BM67HT60,Xtrackers MSCI World Information Tech UCITS ET...,EUR,Xetra
198,IE00BL25JP72,Xtrackers MSCI World Momentum UCITS ETF 1C,EUR,Xetra


In [ ]:
bolsas = free_ETF['Bolsa']
bolsas = bolsas.explode().unique()
bolsas
dict_bolsas = {'Euronext Paris':'france', 'Bolsa de Madrid':'spain', 'Xetra':'germany',
       'Borsa Italiana S.p.A.':'italy', 'Euronext Amsterdam':'netherlands',
       'London Stock Exchange':'united kingdom', 'Euronext Brussels':'belgium'}

In [ ]:
free_ETF['Pais'] = free_ETF['Bolsa'].map(dict_bolsas)
paises = free_ETF['Pais']
paises = paises.explode().unique()
paises = paises.tolist()
paises

['france',
 'spain',
 'germany',
 'italy',
 'netherlands',
 'united kingdom',
 'belgium']

#**2. Obter dados de diferentes fontes**

Fontes de dados de ETFs e formas de acesso:


*   Investpy (https://investpy.readthedocs.io/): nome, país
*   Financial Modeling Prep (free só US) (https://financialmodelingprep.com/): código



In [ ]:
#instalar Investpy
!pip install investpy
import investpy
#obter países do Investpy
paises_dados = investpy.etfs.get_etf_countries()

In [ ]:
#obter dados Investpy
dados_investpy = pd.DataFrame()
for pais in paises:
  if pais in paises_dados:
    linha = investpy.etfs.etfs_as_df(country=pais)
    dados_investpy = dados_investpy.append(linha, ignore_index=True)
  else:
    print('País ',pais,'não disponível!')
dados_investpy

,country,name,full_name,symbol,isin,asset_class,currency,stock_exchange,def_stock_exchange
0,france,Lyxor UCITS CAC 40 Daily Double Short,Lyxor UCITS CAC 40 Daily Double Short,BX4,FR0010411884,equity,EUR,Paris,True
1,france,ETFS Natural Gas,ETFS Natural Gas,NGASP,GB00B15KY104,commodity,EUR,Paris,False
2,france,Lyxor UCITS FTSE ATHEX Large Cap,Lyxor UCITS FTSE ATHEX Large Cap,LGRE,FR0010405431,equity,EUR,Paris,True
3,france,Amundi ETF MSCI Emerging Markets UCITS USD,Amundi Index Solutions - Amundi MSCI Emerging ...,AUEM,LU1681045453,equity,USD,Paris,True
4,france,Lyxor UCITS CAC 40 (DR) D-EUR,Lyxor UCITS CAC 40 (DR) D-EUR,CAC,FR0007052782,equity,EUR,Paris,True
...,...,...,...,...,...,...,...,...,...
4919,united kingdom,db x-trackers US Dollar Cash,db x-trackers US Dollar Cash UCITS 1C,XUSD,LU0356591882,other,USD,London,True
4920,united kingdom,Bitcoin Tracker EUR XBT Provider,XBT Provider AB Call Bitcoin 31Dec99,0PTN,SE0007525332,other,EUR,London,False
4921,united kingdom,Bitcoin Tracker One XBT,Bitcoin Tracker One XBT Provider,O2XU,SE0007126024,other,SEK,London,True
4922,belgium,Lyxor UCITS BEL 20 TR,Lyxor UCITS BEL 20 TR,BEL,FR0000021842,equity,EUR,Brussels,True


In [ ]:
#fusão dos ETFs gratuitos com dados do Investpy
merged_investpy = free_ETF.merge(dados_investpy, left_on=['ISIN','Pais'], right_on=['isin','country'])
merged_investpy.head()

,ISIN,Nome,Moeda,Bolsa,Pais,country,name,full_name,symbol,isin,asset_class,currency,stock_exchange,def_stock_exchange
0,LU1681046931,AMUNDI ETF CAC 40,EUR,Euronext Paris,france,france,Amundi CAC 40 UCITS,Amundi CAC 40 UCITS,C40,LU1681046931,equity,EUR,Paris,False
1,LU1681044480,AMUNDI ETF EM ASIA,EUR,Euronext Paris,france,france,Amundi MSCI EM Asia UCITS,Amundi MSCI EM Asia UCITS,AASI,LU1681044480,equity,EUR,Paris,False
2,LU1681047236,AMUNDI ETF ES50 C,EUR,Euronext Paris,france,france,Amundi ETF Euro Stoxx 50 UCITS,Amundi Index Solutions - Amundi EURO STOXX 50 ...,C50,LU1681047236,equity,EUR,Paris,True
3,LU1681045370,AMUNDI ETF MSCI EM,EUR,Euronext Paris,france,france,Amundi ETF MSCI Emerging Markets UCITS,Amundi ETF MSCI Emerging Markets UCITS,AEEM,LU1681045370,equity,EUR,Paris,True
4,LU1681043599,AMUNDI ETF MSCI WR,EUR,Euronext Paris,france,france,Amundi MSCI World UCITS,Amundi MSCI World UCITS,CW8,LU1681043599,equity,EUR,Paris,False


In [ ]:
merged_investpy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 0 to 162
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ISIN                163 non-null    object
 1   Nome                163 non-null    object
 2   Moeda               163 non-null    object
 3   Bolsa               163 non-null    object
 4   Pais                163 non-null    object
 5   country             163 non-null    object
 6   name                163 non-null    object
 7   full_name           163 non-null    object
 8   symbol              163 non-null    object
 9   isin                163 non-null    object
 10  asset_class         163 non-null    object
 11  currency            163 non-null    object
 12  stock_exchange      163 non-null    object
 13  def_stock_exchange  163 non-null    bool  
dtypes: bool(1), object(13)
memory usage: 18.0+ KB


In [ ]:
#ETF com problema
merged_investpy = merged_investpy.drop(110)

In [ ]:
merged_investpy = merged_investpy.drop_duplicates(subset=['ISIN','name'])
merged_investpy

,ISIN,Nome,Moeda,Bolsa,Pais,country,name,full_name,symbol,isin,asset_class,currency,stock_exchange,def_stock_exchange
0,LU1681046931,AMUNDI ETF CAC 40,EUR,Euronext Paris,france,france,Amundi CAC 40 UCITS,Amundi CAC 40 UCITS,C40,LU1681046931,equity,EUR,Paris,False
1,LU1681044480,AMUNDI ETF EM ASIA,EUR,Euronext Paris,france,france,Amundi MSCI EM Asia UCITS,Amundi MSCI EM Asia UCITS,AASI,LU1681044480,equity,EUR,Paris,False
2,LU1681047236,AMUNDI ETF ES50 C,EUR,Euronext Paris,france,france,Amundi ETF Euro Stoxx 50 UCITS,Amundi Index Solutions - Amundi EURO STOXX 50 ...,C50,LU1681047236,equity,EUR,Paris,True
3,LU1681045370,AMUNDI ETF MSCI EM,EUR,Euronext Paris,france,france,Amundi ETF MSCI Emerging Markets UCITS,Amundi ETF MSCI Emerging Markets UCITS,AEEM,LU1681045370,equity,EUR,Paris,True
4,LU1681043599,AMUNDI ETF MSCI WR,EUR,Euronext Paris,france,france,Amundi MSCI World UCITS,Amundi MSCI World UCITS,CW8,LU1681043599,equity,EUR,Paris,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,LU0378818131,Xtrackers II Global Government Bond UCITS ETF 5C,EUR,Xetra,germany,germany,db x-trackers II Global Sovereign UCITS 1C,Xtrackers II Global Government Bond UCITS ETF ...,XGSH,LU0378818131,bond,EUR,Xetra,True
156,LU0514695690,Xtrackers MSCI China UCITS ETF 1C,EUR,Xetra,germany,germany,db x-trackers MSCI China UCITS DR 1C,db x-trackers MSCI China UCITS DR 1C,XCS6,LU0514695690,equity,EUR,Xetra,False
157,IE00BM67HT60,Xtrackers MSCI World Information Tech UCITS ET...,EUR,Xetra,germany,germany,db x-trackers MSCI World Info Tech DR 1C,dbxtrck MSCI Wrld InfoTech DR 1C TG,XDWT,IE00BM67HT60,equity,EUR,Xetra,False
161,IE00BL25JP72,Xtrackers MSCI World Momentum UCITS ETF 1C,EUR,Xetra,germany,germany,db x-trackers MSCI World Mom Factor DR 1C,db x-trackers MSCI World Momentum Factor UCITS...,XDEM,IE00BL25JP72,equity,EUR,Frankfurt,True


#**3. Seleção de Portfolios**

In [ ]:
#obtenção de dados complementares investpy
# seleção apenas Holanda e Alemanha
info_investpy = pd.DataFrame(columns=['ETF Name', 'Prev. Close', 'Todays Range', 'Open', '52 wk Range', 'Volume', 'Average Vol. (3m)', 
                                      '1-Year Change', 'Asset Class'])
linha = {'ETF Name':'ETF Name', 'Prev. Close':'Prev. Close', 'Todays Range':'Todays Range', 'Open':'Open', '52 wk Range':'52 wk Range',
         'Volume':'Volume', 'Average Vol. (3m)':'Average Vol. (3m)','1-Year Change':'1-Year Change','Asset Class':'Asset Class'}

for ETF, row in merged_investpy.iterrows():
  name = merged_investpy.at[ETF,'name']
  country = merged_investpy.at[ETF,'country']
  if country in ['netherlands', 'germany']:
    coluna = str(name)
    df = investpy.etfs.get_etf_information(etf=name,country=country,as_json=False)
    info_investpy = info_investpy.append(df[linha], ignore_index=True)
    print(df[linha])
  else:
    print('Não selecionado: ', name, country)


Não selecionado:  Amundi CAC 40 UCITS france
Não selecionado:  Amundi MSCI EM Asia UCITS france
Não selecionado:  Amundi ETF Euro Stoxx 50 UCITS france
Não selecionado:  Amundi ETF MSCI Emerging Markets UCITS france
Não selecionado:  Amundi MSCI World UCITS france
Não selecionado:  Amundi S&P 500 UCITS france
Não selecionado:  Amundi ETF Stoxx Europe 600 UCITS france
Não selecionado:  Amundi ETF MSCI Europe UCITS france
Não selecionado:  Amundi ETF CAC 40 UCITS france
Não selecionado:  Accion IBEX 35 Cotizado Armonizado FI spain
                             ETF Name  Prev. Close  ... 1-Year Change  Asset Class
0  ComStage NYSE Arca Gold BUGS UCITS        26.22  ...       - 2.07%       Equity

[1 rows x 9 columns]
                        ETF Name  Prev. Close  ... 1-Year Change  Asset Class
0  db x-trackers CSI300 UCITS 1C        15.71  ...        41.46%       Equity

[1 rows x 9 columns]
Não selecionado:  db x-trackers Nikkei 225 UCITS DR italy
Não selecionado:  db x-trackers MSCI Wd H

,ETF Name,Prev. Close,Todays Range,Open,52 wk Range,Volume,Average Vol. (3m),1-Year Change,Asset Class
0,ComStage NYSE Arca Gold BUGS UCITS,26.22,26.41 - 26.73,26.550,20.45 - 31.09,51127,60957.0,- 2.07%,Equity
1,db x-trackers CSI300 UCITS 1C,15.71,15.63 - 15.73,15.680,10.72 - 17.83,87668,106058.0,41.46%,Equity
2,iShares Diversified Commodity Swap UCITS DE,23.67,23.84 - 23.95,23.840,15.98 - 24.05,2649,2985.0,49.6%,Commodity
3,Deka STOXX Europe Strong Growth 20 UCITS,54.37,54.25 - 54.73,54.370,42.24 - 58.79,4259,19290.0,31.92%,Equity
4,Source S&P 500 Hedged,33.24,33.36 - 33.55,33.550,23.32 - 33.98,None,1099.0,46.38%,Equity
...,...,...,...,...,...,...,...,...,...
97,db x-trackers II Global Sovereign UCITS 1C,235.92,235.18 - 235.63,235.420,235.15 - 247.02,242,3451.0,- 3.28%,NaN
98,db x-trackers MSCI China UCITS DR 1C,19.40,19.45 - 19.58,19.580,15.86 - 24.4,271018,153392.0,18.05%,Equity
99,db x-trackers MSCI World Info Tech DR 1C,43.83,44.03 - 44.35,44.315,33.565 - 47.755,9413,65318.0,31.82%,Equity
100,db x-trackers MSCI World Mom Factor DR 1C,41.23,41.57 - 41.77,41.610,33.19 - 44.74,120,440.0,21.64%,Equity


In [ ]:
merged_investpy.at[0:'iShares Core Euro Corporate Bond UCITS']

TypeError: ignored

In [ ]:
#seleção dos 80 mais líquidos
maiores_volumes = info_investpy.sort_values(by=['Average Vol. (3m)'], ascending=False)[:80]
maiores_volumes.reset_index(drop=True, inplace=True)
maiores_volumes

,ETF Name,Prev. Close,Todays Range,Open,52 wk Range,Volume,Average Vol. (3m),1-Year Change,Asset Class
0,iShares Global Clean Energy UCITS,10.014,10.122 - 10.198,10.124,5.862 - 16.07,496428,2335348.0,75.68%,Equity
1,iShares MSCI World Small Cap UCITS USD Acc,5.920,5.93 - 5.97,5.970,4.03 - 6.12,106087,645222.0,51.74%,Equity
2,iShares MSCI China A UCITS USD,5.251,5.22 - 5.261,5.261,3.754 - 6.019,268315,623881.0,34.61%,Equity
3,iShares China CNY Bond UCITS USD Dist,5.500,5.5 - 5.52,5.510,4.95 - 5.52,215506,523344.0,7.7%,NaN
4,iShares STOXX Europe 600 UCITS,44.180,44.3 - 44.58,44.520,33.3 - 44.58,86751,500711.0,35.1%,Equity
...,...,...,...,...,...,...,...,...,...
75,iShares MSCI Brazil UCITS Dist,23.790,23.82 - 23.99,23.940,16.63 - 25.25,453,8010.0,48.13%,Equity
76,Vanguard Eurozone Government Bond,26.640,26.61 - 26.66,26.630,26.61 - 28,5146,7739.0,0.06%,NaN
77,Vanguard FTSE Developed Asia Pacif,26.030,26.23 - 26.44,26.440,18.57 - 27.14,260,7736.0,40.58%,Equity
78,SPDR MSCI World Health Care UCITS,43.360,43.18 - 43.47,43.470,37.67 - 44.12,1465,7275.0,8.15%,Equity


In [ ]:
#fusão dos maiores volumes com dados do Investpy
 
merged_maiores_volumes = maiores_volumes.merge(merged_investpy, left_on=['ETF Name'], right_on=['name'], how='left')
merged_maiores_volumes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ETF Name            80 non-null     object 
 1   Prev. Close         80 non-null     float64
 2   Todays Range        80 non-null     object 
 3   Open                80 non-null     float64
 4   52 wk Range         80 non-null     object 
 5   Volume              80 non-null     object 
 6   Average Vol. (3m)   80 non-null     float64
 7   1-Year Change       80 non-null     object 
 8   Asset Class         72 non-null     object 
 9   ISIN                80 non-null     object 
 10  Nome                80 non-null     object 
 11  Moeda               80 non-null     object 
 12  Bolsa               80 non-null     object 
 13  Pais                80 non-null     object 
 14  country             80 non-null     object 
 15  name                80 non-null     object 
 16  full_name 

In [ ]:
#obtenção do fechamento dos últimos 10 anos
start = '12/05/2011'
end = '12/05/2021'
fechamentos = pd.DataFrame()
for ETF, row in merged_maiores_volumes.iterrows():
  name = merged_maiores_volumes.at[ETF,'name']
  country = merged_maiores_volumes.at[ETF,'country']
  coluna = str(name)
  df = investpy.etfs.get_etf_historical_data(etf=name,country=country,from_date=start,to_date=end)
  fechamentos[coluna] = df['Close']
  print(coluna)
 
fechamentos

/usr/local/lib/python3.7/dist-packages/investpy/etfs.py:591: Warning: Selected country does not contain the default stock exchange of the introduced ETF. Default country is: "united kingdom" and default stock_exchange: "London".
  Warning


iShares Global Clean Energy UCITS
iShares MSCI World Small Cap UCITS USD Acc


/usr/local/lib/python3.7/dist-packages/investpy/etfs.py:591: Warning: Selected country does not contain the default stock exchange of the introduced ETF. Default country is: "united kingdom" and default stock_exchange: "London".
  Warning


iShares MSCI China A UCITS USD
iShares China CNY Bond UCITS USD Dist
iShares STOXX Europe 600 UCITS


/usr/local/lib/python3.7/dist-packages/investpy/etfs.py:591: Warning: Selected country does not contain the default stock exchange of the introduced ETF. Default country is: "italy" and default stock_exchange: "Milan".
  Warning
/usr/local/lib/python3.7/dist-packages/investpy/etfs.py:606: Warning: Note that the displayed information can differ depending on the stock exchange. Available stock_exchange values for "germany" are: "Xetra", "TradeGate", "Frankfurt", "Stuttgart", "Munich".
  Warning


...

/usr/local/lib/python3.7/dist-packages/investpy/etfs.py:591: Warning: Selected country does not contain the default stock exchange of the introduced ETF. Default country is: "united kingdom" and default stock_exchange: "London".
  Warning


iShares MSCI Brazil UCITS Dist
Vanguard Eurozone Government Bond
Vanguard FTSE Developed Asia Pacif


/usr/local/lib/python3.7/dist-packages/investpy/etfs.py:591: Warning: Selected country does not contain the default stock exchange of the introduced ETF. Default country is: "united kingdom" and default stock_exchange: "London".
  Warning


SPDR MSCI World Health Care UCITS
Think Morningstar High Dividend


,iShares Global Clean Energy UCITS,iShares MSCI World Small Cap UCITS USD Acc,iShares MSCI China A UCITS USD,iShares China CNY Bond UCITS USD Dist,iShares STOXX Europe 600 UCITS,iShares Automation & Robotics Acc,iShares Digital Security UCITS Acc,iShares Digitalisation Acc,iShares Nasdaq US Biotechnology UCITS USD (Acc),iShares S&P 500 USD Info Tech,db x-trackers MSCI China UCITS DR 1C,iShares MSCI Europe UCITS Dist,iShares S&P Gold Producers UCITS,iShares Diverisified Commodity SWAP UCITS USD,iShares STOXX Europe 600 Oil & Gas UCITS,iShares STOXX Europe 600 Travel & Leisure UCITS,iShares Core MSCI World UCITS,iShares MSCI China UCITS ETF USD Acc,db x-trackers CSI300 UCITS 1C,iShares MSCI World Momentum Factor UCITS,iShares MSCI USA ESG Screened UCITS Acc,Vanguard S&P 500 UCITS USD Inc,iShares S&P 500 UCITS Dist,iShares STOXX Global Select Dividend 100 UCITS,iShares Core FTSE 100 UCITS,Vanguard FTSE Developed Europe,iShares Core MSCI Emerging Markets IMI UCITS,iShares S&P Commodity Producers Oil and Gas UCITS,db x-trackers MSCI World Info Tech DR 1C,ComStage NYSE Arca Gold BUGS UCITS,iShares NASDAQ-100 UCITS,SSgA SPDR S&P Euro Dividend Aristocrats,Lyxor UCITS New Energy D-EUR,Vanguard FTSE All-World UCITS USD Acc,UBS ETF MSCI World Socially Responsible UCITS USD,iShares TecDAX UCITS,iShares Global Water UCITS,iShares Euro High Yield Corporate Bond UCITS,Vanguard FTSE All-World UCITS USD Inc,iShares EURO Dividend UCITS,iShares AEX,iShares MSCI World UCITS Dist,KraneShares CSI China Internet,iShares Core S&P 500 UCITS,iShares European Property Yield UCITS,iShares Listed Private Equity UCITS,iShares MSCI Turkey UCITS,iShares MSCI Emerging Markets UCITS Dist,iShares MSCI Japan UCITS Dist,SPDR Russell 2000 U.S. Small Cap UCITS,iShares EURO STOXX 50 UCITS Dist,iShares Developed Markets Property Yield UCITS,iShares MSCI World EUR Hedged UCITS,iShares MSCI Europe UCITS Acc,db x-trackers II iBoxx EUR High,db x-trackers USD Corporate Bond DR 2D Hedged,Deka STOXX Europe Strong Growth 20 UCITS,iShares Asia Pacific Dividend UCITS,iShares STOXX Europe Select Dividend 30 UCITS,Think AEX,Lyxor UCITS World Water D-EUR,Vanguard FTSE All-World High Div Yld,iShares Core Euro Corporate Bond UCITS,iShares S&P 500 EUR Hedged UCITS,iShares Global Infrastructure UCITS,db x-trackers II iBoxx Liquid Corporate Financials,db x-trackers II Barclays Global Aggregate Bond UC,PowerShares SP 500 High Dividend FR,iShares Bric 50 UCITS,Vanguard FTSE Emerging Markets UCITS USD Inc,iShares MSCI Korea UCITS Dist,SPDR MSCI World Technology,Think Global Equity,iShares MSCI Emerging Markets UCITS Acc,iShares China Large Cap UCITS,iShares MSCI Brazil UCITS Dist,Vanguard Eurozone Government Bond,Vanguard FTSE Developed Asia Pacif,SPDR MSCI World Health Care UCITS,Think Morningstar High Dividend
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-05-12,6.980,NaN,NaN,NaN,28.80,NaN,NaN,NaN,NaN,NaN,NaN,17.98,NaN,NaN,33.50,13.28,21.34,NaN,NaN,NaN,NaN,NaN,NaN,20.53,6.85,NaN,NaN,NaN,NaN,36.54,16.90,NaN,18.95,NaN,NaN,8.84,18.11,104.63,NaN,19.51,36.40,20.22,NaN,80.94,28.31,10.96,27.69,31.88,7.33,NaN,29.66,14.53,27.89,30.52,NaN,NaN,16.45,21.77,15.83,36.42,17.49,NaN,115.65,29.48,16.52,119.02,NaN,NaN,22.03,NaN,28.56,NaN,24.55,22.34,86.82,41.62,NaN,NaN,NaN,NaN
2011-05-13,6.970,NaN,NaN,NaN,28.76,NaN,NaN,NaN,NaN,NaN,NaN,17.93,NaN,NaN,33.35,13.27,21.36,NaN,NaN,NaN,NaN,NaN,NaN,20.59,6.82,NaN,NaN,NaN,NaN,36.42,16.92,NaN,18.87,NaN,NaN,8.91,18.08,104.47,NaN,19.43,36.15,20.23,NaN,81.39,28.41,10.96,26.57,31.74,7.28,NaN,29.48,14.54,27.87,30.45,NaN,NaN,16.46,21.70,15.76,36.19,17.42,NaN,115.75,29.50,16.50,118.95,NaN,NaN,21.93,NaN,28.39,NaN,NaN,22.23,86.96,40.86,NaN,NaN,NaN,NaN
2011-05-16,6.930,NaN,NaN,NaN,28.70,NaN,NaN,NaN,NaN,NaN,NaN,17.91,NaN,NaN,33.21,13.22,21.29,NaN,NaN,NaN,NaN,NaN,NaN,20.63,6.81,NaN,NaN,NaN,NaN,36.86,16.74,NaN,18.73,NaN,NaN,8.88,17.92,104.56,NaN,19.34,35.88,20.18,NaN,81.07,28.15,10.86,27.21,31.67,7.21,NaN,29.39,14.55,27.84,30.38,NaN,Na

In [ ]:
fechamentos = fechamentos.dropna(how='all')
fechamentos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2532 entries, 2011-05-12 to 2021-05-12
Data columns (total 80 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   iShares Global Clean Energy UCITS                   2532 non-null   float64
 1   iShares MSCI World Small Cap UCITS USD Acc          767 non-null    float64
 2   iShares MSCI China A UCITS USD                      1536 non-null   float64
 3   iShares China CNY Bond UCITS USD Dist               451 non-null    float64
 4   iShares STOXX Europe 600 UCITS                      2532 non-null   float64
 5   iShares Automation & Robotics Acc                   1174 non-null   float64
 6   iShares Digital Security UCITS Acc                  668 non-null    float64
 7   iShares Digitalisation Acc                          1173 non-null   float64
 8   iShares Nasdaq US Biotechnology UCITS USD (Acc)     871 non-

In [ ]:
# corte dos que não têm pelo menos 5 anos de histórico
corte = fechamentos
corte = corte[corte.columns].count().map(lambda x: x<1260)
corte = corte[corte==False]
corte = corte.index.to_list()
corte

fechamentos = fechamentos[corte]


fechamentos

,iShares Global Clean Energy UCITS,iShares MSCI China A UCITS USD,iShares STOXX Europe 600 UCITS,iShares S&P 500 USD Info Tech,db x-trackers MSCI China UCITS DR 1C,iShares MSCI Europe UCITS Dist,iShares S&P Gold Producers UCITS,iShares STOXX Europe 600 Oil & Gas UCITS,iShares STOXX Europe 600 Travel & Leisure UCITS,iShares Core MSCI World UCITS,db x-trackers CSI300 UCITS 1C,iShares MSCI World Momentum Factor UCITS,Vanguard S&P 500 UCITS USD Inc,iShares S&P 500 UCITS Dist,iShares STOXX Global Select Dividend 100 UCITS,iShares Core FTSE 100 UCITS,Vanguard FTSE Developed Europe,iShares Core MSCI Emerging Markets IMI UCITS,iShares S&P Commodity Producers Oil and Gas UCITS,db x-trackers MSCI World Info Tech DR 1C,ComStage NYSE Arca Gold BUGS UCITS,iShares NASDAQ-100 UCITS,SSgA SPDR S&P Euro Dividend Aristocrats,Lyxor UCITS New Energy D-EUR,UBS ETF MSCI World Socially Responsible UCITS USD,iShares TecDAX UCITS,iShares Global Water UCITS,iShares Euro High Yield Corporate Bond UCITS,Vanguard FTSE All-World UCITS USD Inc,iShares EURO Dividend UCITS,iShares AEX,iShares MSCI World UCITS Dist,iShares Core S&P 500 UCITS,iShares European Property Yield UCITS,iShares Listed Private Equity UCITS,iShares MSCI Turkey UCITS,iShares MSCI Emerging Markets UCITS Dist,iShares MSCI Japan UCITS Dist,SPDR Russell 2000 U.S. Small Cap UCITS,iShares EURO STOXX 50 UCITS Dist,iShares Developed Markets Property Yield UCITS,iShares MSCI World EUR Hedged UCITS,iShares MSCI Europe UCITS Acc,db x-trackers II iBoxx EUR High,db x-trackers USD Corporate Bond DR 2D Hedged,Deka STOXX Europe Strong Growth 20 UCITS,iShares Asia Pacific Dividend UCITS,iShares STOXX Europe Select Dividend 30 UCITS,Think AEX,Lyxor UCITS World Water D-EUR,Vanguard FTSE All-World High Div Yld,iShares Core Euro Corporate Bond UCITS,iShares S&P 500 EUR Hedged UCITS,iShares Global Infrastructure UCITS,db x-trackers II iBoxx Liquid Corporate Financials,db x-trackers II Barclays Global Aggregate Bond UC,iShares Bric 50 UCITS,Vanguard FTSE Emerging Markets UCITS USD Inc,iShares MSCI Korea UCITS Dist,SPDR MSCI World Technology,Think Global Equity,iShares MSCI Emerging Markets UCITS Acc,iShares China Large Cap UCITS,iShares MSCI Brazil UCITS Dist,Vanguard Eurozone Government Bond,Vanguard FTSE Developed Asia Pacif
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-05-12,6.980,NaN,28.80,NaN,NaN,17.98,NaN,33.50,13.28,21.34,NaN,NaN,NaN,NaN,20.53,6.85,NaN,NaN,NaN,NaN,36.54,16.90,NaN,18.95,NaN,8.84,18.11,104.63,NaN,19.51,36.40,20.22,80.94,28.31,10.96,27.69,31.88,7.33,NaN,29.66,14.53,27.89,30.52,NaN,NaN,16.45,21.77,15.83,36.42,17.49,NaN,115.65,29.48,16.52,119.02,NaN,22.03,NaN,28.56,NaN,24.55,22.34,86.82,41.62,NaN,NaN
2011-05-13,6.970,NaN,28.76,NaN,NaN,17.93,NaN,33.35,13.27,21.36,NaN,NaN,NaN,NaN,20.59,6.82,NaN,NaN,NaN,NaN,36.42,16.92,NaN,18.87,NaN,8.91,18.08,104.47,NaN,19.43,36.15,20.23,81.39,28.41,10.96,26.57,31.74,7.28,NaN,29.48,14.54,27.87,30.45,NaN,NaN,16.46,21.70,15.76,36.19,17.42,NaN,115.75,29.50,16.50,118.95,NaN,21.93,NaN,28.39,NaN,NaN,22.23,86.96,40.86,NaN,NaN
2011-05-16,6.930,NaN,28.70,NaN,NaN,17.91,NaN,33.21,13.22,21.29,NaN,NaN,NaN,NaN,20.63,6.81,NaN,NaN,NaN,NaN,36.86,16.74,NaN,18.73,NaN,8.88,17.92,104.56,NaN,19.34,35.88,20.18,81.07,28.15,10.86,27.21,31.67,7.21,NaN,29.39,14.55,27.84,30.38,NaN,NaN,16.43,21.88,15.70,35.90,17.36,NaN,115.74,29.48,16.45,118.85,NaN,22.01,NaN,28.21,NaN,24.43,22.21,86.58,41.25,NaN,NaN
2011-05-17,6.820,NaN,28.45,NaN,NaN,17.73,NaN,32.98,13.10,21.05,NaN,NaN,NaN,NaN,20.51,6.76,NaN,NaN,NaN,NaN,36.12,16.50,NaN,18.57,NaN,8.78,17.84,104.55,NaN,19.20,35.42,19.98,80.29,27.98,10.65,27.00,31.41,7.12,NaN,29.09,14.44,27.44,30.12,NaN,NaN,16.25,21.66,15.62,35.45,17.24,NaN,115.79,29.00,16.37,118.88,NaN,21.75,NaN,28.00,NaN,24.37,22.02,86.26,40.70,NaN,NaN
2011-05-18,6.840,NaN,28.47,NaN,NaN,17.78,NaN,33.25,13.02,21.16,NaN,NaN,NaN,NaN,20.48,6.76,NaN,NaN,NaN,NaN,36.91,16.56,NaN,18.64,NaN,8.80,17.84,104.51,NaN,19.23,35.55,20.05,80.49,28.37,10.71,26.57,31.84,7.18,NaN,2

In [ ]:
fechamentos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2532 entries, 2011-05-12 to 2021-05-12
Data columns (total 66 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   iShares Global Clean Energy UCITS                   2532 non-null   float64
 1   iShares MSCI China A UCITS USD                      1536 non-null   float64
 2   iShares STOXX Europe 600 UCITS                      2532 non-null   float64
 3   iShares S&P 500 USD Info Tech                       1368 non-null   float64
 4   db x-trackers MSCI China UCITS DR 1C                2462 non-null   float64
 5   iShares MSCI Europe UCITS Dist                      2532 non-null   float64
 6   iShares S&P Gold Producers UCITS                    2299 non-null   float64
 7   iShares STOXX Europe 600 Oil & Gas UCITS            2526 non-null   float64
 8   iShares STOXX Europe 600 Travel & Leisure UCITS     2505 non

#**4. Backtest de Portfolios**

In [ ]:
!pip install PyPortfolioOpt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
expected_mean = expected_returns.mean_historical_return(fechamentos)
expected_mean = expected_mean/expected_mean.max()
expected_mean = expected_mean.sort_values(ascending=False)
expected_mean

SPDR MSCI World Technology                           1.000000
db x-trackers MSCI World Info Tech DR 1C             0.913704
iShares S&P 500 USD Info Tech                        0.861477
iShares NASDAQ-100 UCITS                             0.778694
iShares Core S&P 500 UCITS                           0.603601
                                                       ...   
iShares STOXX Europe 600 Oil & Gas UCITS            -0.094930
ComStage NYSE Arca Gold BUGS UCITS                  -0.148414
iShares S&P Commodity Producers Oil and Gas UCITS   -0.185738
iShares MSCI Brazil UCITS Dist                      -0.210311
iShares MSCI Turkey UCITS                           -0.371264
Length: 66, dtype: float64

In [ ]:
expected_ema = expected_returns.ema_historical_return(fechamentos)
expected_ema = expected_ema/expected_ema.max()
expected_ema = expected_ema.sort_values(ascending=False)
expected_ema

iShares S&P Commodity Producers Oil and Gas UCITS     1.000000
iShares Global Clean Energy UCITS                     0.926642
iShares Listed Private Equity UCITS                   0.850202
iShares MSCI Korea UCITS Dist                         0.846958
SPDR Russell 2000 U.S. Small Cap UCITS                0.842060
                                                        ...   
iShares China Large Cap UCITS                        -0.017523
Vanguard Eurozone Government Bond                    -0.024616
db x-trackers II Barclays Global Aggregate Bond UC   -0.081961
db x-trackers USD Corporate Bond DR 2D Hedged        -0.109906
iShares MSCI Turkey UCITS                            -0.354762
Name: 2021-05-12 00:00:00, Length: 66, dtype: float64

In [ ]:
expected_capm = expected_returns.capm_return(fechamentos)
expected_capm = expected_capm/expected_capm.max()
expected_capm = expected_capm.sort_values(ascending=False)
expected_capm

iShares MSCI Brazil UCITS Dist                        1.000000
iShares S&P Commodity Producers Oil and Gas UCITS     0.962030
iShares Core MSCI Emerging Markets IMI UCITS          0.879456
iShares Global Clean Energy UCITS                     0.877005
iShares Bric 50 UCITS                                 0.873802
                                                        ...   
db x-trackers USD Corporate Bond DR 2D Hedged         0.307671
db x-trackers II iBoxx Liquid Corporate Financials    0.271120
db x-trackers II Barclays Global Aggregate Bond UC    0.265859
iShares Core Euro Corporate Bond UCITS                0.263794
Vanguard Eurozone Government Bond                     0.235861
Name: mkt, Length: 66, dtype: float64

In [ ]:
risk_samp_cov = risk_models.risk_matrix(fechamentos,method='sample_cov')
#risk_exp_covs = risk_exp_covs.ort_values(ascending=False).iloc[:20]
risk_samp_cov

/usr/local/lib/python3.7/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."


,iShares Global Clean Energy UCITS,iShares MSCI China A UCITS USD,iShares STOXX Europe 600 UCITS,iShares S&P 500 USD Info Tech,db x-trackers MSCI China UCITS DR 1C,iShares MSCI Europe UCITS Dist,iShares S&P Gold Producers UCITS,iShares STOXX Europe 600 Oil & Gas UCITS,iShares STOXX Europe 600 Travel & Leisure UCITS,iShares Core MSCI World UCITS,db x-trackers CSI300 UCITS 1C,iShares MSCI World Momentum Factor UCITS,Vanguard S&P 500 UCITS USD Inc,iShares S&P 500 UCITS Dist,iShares STOXX Global Select Dividend 100 UCITS,iShares Core FTSE 100 UCITS,Vanguard FTSE Developed Europe,iShares Core MSCI Emerging Markets IMI UCITS,iShares S&P Commodity Producers Oil and Gas UCITS,db x-trackers MSCI World Info Tech DR 1C,ComStage NYSE Arca Gold BUGS UCITS,iShares NASDAQ-100 UCITS,SSgA SPDR S&P Euro Dividend Aristocrats,Lyxor UCITS New Energy D-EUR,UBS ETF MSCI World Socially Responsible UCITS USD,iShares TecDAX UCITS,iShares Global Water UCITS,iShares Euro High Yield Corporate Bond UCITS,Vanguard FTSE All-World UCITS USD Inc,iShares EURO Dividend UCITS,iShares AEX,iShares MSCI World UCITS Dist,iShares Core S&P 500 UCITS,iShares European Property Yield UCITS,iShares Listed Private Equity UCITS,iShares MSCI Turkey UCITS,iShares MSCI Emerging Markets UCITS Dist,iShares MSCI Japan UCITS Dist,SPDR Russell 2000 U.S. Small Cap UCITS,iShares EURO STOXX 50 UCITS Dist,iShares Developed Markets Property Yield UCITS,iShares MSCI World EUR Hedged UCITS,iShares MSCI Europe UCITS Acc,db x-trackers II iBoxx EUR High,db x-trackers USD Corporate Bond DR 2D Hedged,Deka STOXX Europe Strong Growth 20 UCITS,iShares Asia Pacific Dividend UCITS,iShares STOXX Europe Select Dividend 30 UCITS,Think AEX,Lyxor UCITS World Water D-EUR,Vanguard FTSE All-World High Div Yld,iShares Core Euro Corporate Bond UCITS,iShares S&P 500 EUR Hedged UCITS,iShares Global Infrastructure UCITS,db x-trackers II iBoxx Liquid Corporate Financials,db x-trackers II Barclays Global Aggregate Bond UC,iShares Bric 50 UCITS,Vanguard FTSE Emerging Markets UCITS USD Inc,iShares MSCI Korea UCITS Dist,SPDR MSCI World Technology,Think Global Equity,iShares MSCI Emerging Markets UCITS Acc,iShares China Large Cap UCITS,iShares MSCI Brazil UCITS Dist,Vanguard Eurozone Government Bond,Vanguard FTSE Developed Asia Pacif
iShares Global Clean Energy UCITS,0.061553,0.037730,0.030792,0.036457,0.037755,0.031085,0.010860,0.036567,0.031363,0.029971,0.033296,0.033922,0.031460,0.030539,0.027135,0.031319,0.031206,0.039028,0.042563,0.034522,0.012008,0.031764,0.026146,0.042958,0.027563,0.032637,0.027427,0.008366,0.031801,0.030673,0.030764,0.030243,0.028431,0.023225,0.035483,0.028400,0.036327,0.027315,0.039790,0.034317,0.024531,0.028285,0.030744,0.010901,0.005448,0.030673,0.029437,0.028983,0.030718,0.027518,0.029376,0.002330,0.027070,0.023810,0.002863,0.002354,0.039254,0.036838,0.035050,0.034352,0.029277,0.035975,0.037590,0.046363,0.001030,0.033972
iShares MSCI China A UCITS USD,0.037730,0.092047,0.026305,0.027370,0.055149,0.026395,0.005404,0.030737,0.024125,0.026712,0.086558,0.029318,0.028133,0.027335,0.024652,0.028053,0.027165,0.042265,0.038616,0.024802,0.005035,0.029943,0.020468,0.029535,0.025147,0.026084,0.023541,0.005894,0.029525,0.023811,0.025721,0.027004,0.026350,0.017210,0.027866,0.023904,0.039929,0.027065,0.030164,0.027281,0.019544,0.022825,0.026312,0.006835,0.001614,0.026037,0.030618,0.023802,0.025825,0.024817,0.027746,0.001814,0.021154,0.018473,0.002204,0.002074,0.046891,0.040847,0.035898,0.024659,0.026818,0.039775,0.053575,0.039332,0.000049,0.036625
iShares STOXX Europe 600 UCITS,0.030792,0.026305,0.030543,0.027488,0.027503,0.029965,0.004653,0.033800,0.028608,0.025024,0.023297,0.025709,0.024990,0.024054,0.023343,0.030092,0.029627,0.029699,0.034143,0.025681,0.005637,0.023673,0.025495,0.029185,0.022690,0.028721,0.022671,0.007458,0.025993,0.031053,0.029431,0.025313,0.022595,0.022568,0.028734,0.025071,0.028054,0.021958,0.030182,0.034188,0.020081,0.024585,0.029567,0.009255,0.003370,0.026587,0.024194,0.028181,0

In [ ]:
#estimar papeis com menores covariâncias
menores_variancias = risk_samp_cov.sum()
menores_variancias = menores_variancias/menores_variancias.max()
menores_variancias = menores_variancias.sort_values()
menores_variancias

Vanguard Eurozone Government Bond                     0.013757
db x-trackers II Barclays Global Aggregate Bond UC    0.051341
iShares Core Euro Corporate Bond UCITS                0.055163
db x-trackers II iBoxx Liquid Corporate Financials    0.067014
db x-trackers USD Corporate Bond DR 2D Hedged         0.105987
                                                        ...   
iShares Core MSCI Emerging Markets IMI UCITS          0.828973
iShares Bric 50 UCITS                                 0.830518
iShares Global Clean Energy UCITS                     0.838798
iShares S&P Commodity Producers Oil and Gas UCITS     0.961594
iShares MSCI Brazil UCITS Dist                        1.000000
Length: 66, dtype: float64

In [ ]:
#deprecated
list0 = set(menores_variancias.index[:30])
list1 = set(expected_capm.index[:60])
list2 = set(expected_ema.index[:60])
list3 = set(expected_mean.index[:60])
selecao1 = set.intersection(list0, list1)
selecao2 = set.intersection(list0, list2)
selecao3 = set.intersection(list0, list3)

#print(list0, list1, list2, list3)

In [ ]:
mean_expected = (expected_mean+expected_ema+expected_capm)/3
indice_ricardo = mean_expected**1/2/menores_variancias**2
indice_ricardo = indice_ricardo.sort_values(ascending=False)
#indice_ricardo = indice_ricardo.head(50)
indice_ricardo

Vanguard Eurozone Government Bond                     230.165031
iShares Core Euro Corporate Bond UCITS                 18.958124
db x-trackers II Barclays Global Aggregate Bond UC     17.991840
db x-trackers II iBoxx Liquid Corporate Financials     14.106696
db x-trackers USD Corporate Bond DR 2D Hedged           3.037280
                                                         ...    
iShares Bric 50 UCITS                                   0.288175
iShares STOXX Europe 600 Oil & Gas UCITS                0.250348
iShares China Large Cap UCITS                           0.234520
iShares MSCI Brazil UCITS Dist                          0.189037
iShares MSCI Turkey UCITS                               0.013608
Length: 66, dtype: float64

In [ ]:
selecao_final = indice_ricardo.index[:8]
selecao_final

Index(['Vanguard Eurozone Government Bond',
       'iShares Core Euro Corporate Bond UCITS',
       'db x-trackers II Barclays Global Aggregate Bond UC',
       'db x-trackers II iBoxx Liquid Corporate Financials',
       'db x-trackers USD Corporate Bond DR 2D Hedged',
       'iShares S&P Gold Producers UCITS',
       'ComStage NYSE Arca Gold BUGS UCITS',
       'iShares Euro High Yield Corporate Bond UCITS'],
      dtype='object')

In [ ]:
#deprecated
covar = set(indice_ricardo.index[:50])
escolhidos = set.intersection(covar,list0)
escolhidos1 = set.intersection(covar, selecao1)
escolhidos2 = set.intersection(covar, selecao2)
escolhidos3 = set.intersection(covar, selecao3)

In [ ]:
#deprecated
selecao_final = set.intersection(escolhidos1,escolhidos2,escolhidos3)
selecao_ricardo = set(indice_ricardo.index[:10])
selecao_final = selecao_final.union(selecao_ricardo,selecao_final)
selecao_final

{'Amundi S&P 500 UCITS',
 'ComStage NYSE Arca Gold BUGS UCITS',
 'Deka STOXX Europe Strong Growth 20 UCITS',
 'HSBC MSCI World UCITS',
 'Lyxor UCITS Nasdaq-100 D-EUR',
 'Lyxor UCITS S&P 500 D-EUR',
 'Lyxor UCITS World Water D-EUR',
 'UBS ETF MSCI World Socially Responsible UCITS USD',
 'db x-trackers II Barclays Global Aggregate Bond UC',
 'db x-trackers II iBoxx Liquid Corporate Financials',
 'db x-trackers MSCI Wd Health Care',
 'db x-trackers MSCI World Info Tech DR 1C',
 'db x-trackers Nikkei 225 UCITS DR',
 'iShares Core Euro Corporate Bond UCITS',
 'iShares Core MSCI World UCITS',
 'iShares Core S&P 500 UCITS',
 'iShares Developed Markets Property Yield UCITS',
 'iShares Global Infrastructure UCITS',
 'iShares Global Water UCITS',
 'iShares MSCI World EUR Hedged UCITS',
 'iShares NASDAQ-100 UCITS',
 'iShares S&P 500 EUR Hedged UCITS',
 'iShares S&P 500 USD Info Tech',
 'iShares S&P Gold Producers UCITS',
 'iShares STOXX Global Select Dividend 100 UCITS'}

In [ ]:
selecao_final

Index(['Vanguard Eurozone Government Bond',
       'iShares Core Euro Corporate Bond UCITS',
       'db x-trackers II Barclays Global Aggregate Bond UC',
       'db x-trackers II iBoxx Liquid Corporate Financials',
       'db x-trackers USD Corporate Bond DR 2D Hedged',
       'iShares S&P Gold Producers UCITS',
       'ComStage NYSE Arca Gold BUGS UCITS',
       'iShares Euro High Yield Corporate Bond UCITS'],
      dtype='object')

In [ ]:
merged_maiores_volumes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ETF Name            80 non-null     object 
 1   Prev. Close         80 non-null     float64
 2   Todays Range        80 non-null     object 
 3   Open                80 non-null     float64
 4   52 wk Range         80 non-null     object 
 5   Volume              80 non-null     object 
 6   Average Vol. (3m)   80 non-null     float64
 7   1-Year Change       80 non-null     object 
 8   Asset Class         72 non-null     object 
 9   ISIN                80 non-null     object 
 10  Nome                80 non-null     object 
 11  Moeda               80 non-null     object 
 12  Bolsa               80 non-null     object 
 13  Pais                80 non-null     object 
 14  country             80 non-null     object 
 15  name                80 non-null     object 
 16  full_name 

In [ ]:
dados_finais = merged_maiores_volumes.set_index(keys='ETF Name', inplace=True)
#[selecao_final]
#dados_finais = merged0dados_finais.loc['ETC Name'][selecao_final]
dados_finais

In [ ]:
dados_finais = merged_maiores_volumes.loc[selecao_final]
dados_finais

,Prev. Close,Todays Range,Open,52 wk Range,Volume,Average Vol. (3m),1-Year Change,Asset Class,ISIN,Nome,Moeda,Bolsa,Pais,country,name,full_name,symbol,isin,asset_class,currency,stock_exchange,def_stock_exchange
Vanguard Eurozone Government Bond,26.64,26.61 - 26.66,26.63,26.61 - 28,5146,7739.0,0.06%,NaN,IE00BZ163H91,VANGUARD EURGVBD,EUR,Euronext Amsterdam,netherlands,netherlands,Vanguard Eurozone Government Bond,Vanguard Eurozone Government Bond UCITS,VETY,IE00BZ163H91,bond,EUR,Amsterdam,True
iShares Core Euro Corporate Bond UCITS,134.05,134.05 - 134.32,134.32,128.44 - 137.09,5651,13551.0,4.34%,NaN,IE00B3F81R35,ISHARES B E CORP,EUR,Euronext Amsterdam,netherlands,netherlands,iShares Core Euro Corporate Bond UCITS,iShares Core Euro Corporate Bond UCITS,IEAC,IE00B3F81R35,bond,EUR,Amsterdam,False
db x-trackers II Barclays Global Aggregate Bond UC,40.28,40.1 - 40.21,40.13,40.1 - 44.01,3676,12099.0,- 8.26%,NaN,LU0942970798,Xtrackers II Glbal Aggregate Bnd Swap UCITS ET...,EUR,Xetra,germany,germany,db x-trackers II Barclays Global Aggregate Bon...,db x-trackers II Barclays Global Aggregate Bon...,XBAG,LU0942970798,bond,EUR,Xetra,False
db x-trackers II iBoxx Liquid Corporate Financials,156.86,156.78 - 156.91,156.79,151.93 - 161.4,3696,13054.0,2.99%,NaN,LU0484968812,Xtrackers II ESG EUR Corporate Bond UCITS ETF 1C,EUR,Xetra,germany,germany,db x-trackers II iBoxx Liquid Corporate Financ...,db x-trackers II iBoxx Liquid Corporate Financ...,XB4F,LU0484968812,bond,EUR,Xetra,True
db x-trackers USD Corporate Bond DR 2D Hedged,14.25,14.21 - 14.23,14.22,14.1 - 15.49,274,20214.0,- 4.52%,NaN,IE00BZ036J45,Xtrackers USD Corporate Bond UCITS ETF 1D,EUR,Xetra,germany,germany,db x-trackers USD Corporate Bond DR 2D Hedged,db x-trackers USD Corporate Bond UCITS DR 2D H...,XDGE,IE00BZ036J45,bond,EUR,Xetra,True
iShares S&P Gold Producers UCITS,14.12,14.2 - 14.29,14.25,11.11 - 16.5,238021,147960.0,- 1.94%,Equity,IE00B6R52036,iShares Gold Producers UCITS ETF USD (Acc),EUR,Xetra,germany,germany,iShares S&P Gold Producers UCITS,iShares S&P Gold Producers UCITS,IS0E,IE00B6R52036,equity,EUR,Xetra,True
ComStage NYSE Arca Gold BUGS UCITS,26.22,26.41 - 26.73,26.55,20.45 - 31.09,51127,60957.0,- 2.07%,Equity,LU0488317701,ComStage NYSE Arca Gold BUGS UCITS ETF,EUR,Xetra,germany,germany,ComStage NYSE Arca Gold BUGS UCITS,ComStage NYSE Arca Gold BUGS UCITS ETF,CBHUI,LU0488317701,equity,EUR,Xetra,True
iShares Euro High Yield Corporate Bond UCITS,102.52,102.5 - 102.79,102.79,92.25 - 103.65,10438,39268.0,12.63%,NaN,IE00B66F4759,iShares High Yield Corp Bond UCITS ETF EUR(Dist),EUR,Xetra,germany,germany,iShares Euro High Yield Corporate Bond UCITS,iShares Euro High Yield Corporate Bond UCITS,EUNW,IE00B66F4759,bond,EUR,Xetra,False


In [ ]:
dados_finais.info

<bound method DataFrame.info of                                                     Prev. Close  ... def_stock_exchange
Vanguard Eurozone Government Bond                         26.64  ...               True
iShares Core Euro Corporate Bond UCITS                   134.05  ...              False
db x-trackers II Barclays Global Aggregate Bond UC        40.28  ...              False
db x-trackers II iBoxx Liquid Corporate Financials       156.86  ...               True
db x-trackers USD Corporate Bond DR 2D Hedged             14.25  ...               True
iShares S&P Gold Producers UCITS                          14.12  ...               True
ComStage NYSE Arca Gold BUGS UCITS                        26.22  ...               True
iShares Euro High Yield Corporate Bond UCITS             102.52  ...              False

[8 rows x 22 columns]>

In [ ]:
fechamentos_finais = fechamentos[selecao_final]
fechamentos_finais

,Vanguard Eurozone Government Bond,iShares Core Euro Corporate Bond UCITS,db x-trackers II Barclays Global Aggregate Bond UC,db x-trackers II iBoxx Liquid Corporate Financials,db x-trackers USD Corporate Bond DR 2D Hedged,iShares S&P Gold Producers UCITS,ComStage NYSE Arca Gold BUGS UCITS,iShares Euro High Yield Corporate Bond UCITS
Date,,,,,,,,
2011-05-12,NaN,115.65,NaN,119.02,NaN,NaN,36.54,104.63
2011-05-13,NaN,115.75,NaN,118.95,NaN,NaN,36.42,104.47
2011-05-16,NaN,115.74,NaN,118.85,NaN,NaN,36.86,104.56
2011-05-17,NaN,115.79,NaN,118.88,NaN,NaN,36.12,104.55
2011-05-18,NaN,115.69,NaN,118.84,NaN,NaN,36.91,104.51
...,...,...,...,...,...,...,...,...
2021-05-06,26.96,134.95,40.62,157.67,14.32,13.42,24.72,102.57
2021-05-07,26.89,134.91,40.39,157.68,14.34,13.47,24.62,102.68
2021-05-10,26.91,134.92,40.34,157.69,14.31,13.64,25.02,102.68


In [ ]:
fechamentos_finais.to_csv("/content/drive/MyDrive/Arquivos Python/fechamntos_escolhidos.csv")

In [ ]:
# PyPortfolioOpt - https://pyportfolioopt.readthedocs.io/

In [ ]:
returns = expected_returns.mean_historical_return(fechamentos_finais)
returns

Vanguard Eurozone Government Bond                     0.012863
iShares Core Euro Corporate Bond UCITS                0.014952
db x-trackers II Barclays Global Aggregate Bond UC    0.025827
db x-trackers II iBoxx Liquid Corporate Financials    0.027900
db x-trackers USD Corporate Bond DR 2D Hedged         0.001782
iShares S&P Gold Producers UCITS                     -0.010737
ComStage NYSE Arca Gold BUGS UCITS                   -0.038085
iShares Euro High Yield Corporate Bond UCITS         -0.002075
dtype: float64

In [ ]:
variances = risk_models.risk_matrix(fechamentos_finais, method='sample_cov')
variances

,Vanguard Eurozone Government Bond,iShares Core Euro Corporate Bond UCITS,db x-trackers II Barclays Global Aggregate Bond UC,db x-trackers II iBoxx Liquid Corporate Financials,db x-trackers USD Corporate Bond DR 2D Hedged,iShares S&P Gold Producers UCITS,ComStage NYSE Arca Gold BUGS UCITS,iShares Euro High Yield Corporate Bond UCITS
Vanguard Eurozone Government Bond,0.001595,0.000911,0.000997,0.001047,0.001222,0.002642,0.002988,0.000822
iShares Core Euro Corporate Bond UCITS,0.000911,0.001588,0.000790,0.001317,0.001629,0.001588,0.002100,0.001598
db x-trackers II Barclays Global Aggregate Bond UC,0.000997,0.000790,0.004055,0.000763,0.001773,0.002362,0.001737,0.000591
db x-trackers II iBoxx Liquid Corporate Financials,0.001047,0.001317,0.000763,0.002077,0.002067,0.000989,0.001505,0.001917
db x-trackers USD Corporate Bond DR 2D Hedged,0.001222,0.001629,0.001773,0.002067,0.007015,0.009349,0.010034,0.002956
iShares S&P Gold Producers UCITS,0.002642,0.001588,0.002362,0.000989,0.009349,0.109977,0.117455,0.002454
ComStage NYSE Arca Gold BUGS UCITS,0.002988,0.002100,0.001737,0.001505,0.010034,0.117455,0.133051,0.003136
iShares Euro High Yield Corporate Bond UCITS,0.000822,0.001598,0.000591,0.001917,0.002956,0.002454,0.003136,0.005643


In [ ]:
ef = EfficientFrontier(returns, variances)
ef.min_volatility()

OrderedDict([('Vanguard Eurozone Government Bond', 0.4075743492271228),
             ('iShares Core Euro Corporate Bond UCITS', 0.4110156827786146),
             ('db x-trackers II Barclays Global Aggregate Bond UC',
              0.1032588875588406),
             ('db x-trackers II iBoxx Liquid Corporate Financials',
              0.0764051735682083),
             ('db x-trackers USD Corporate Bond DR 2D Hedged', 0.0),
             ('iShares S&P Gold Producers UCITS', 0.0),
             ('ComStage NYSE Arca Gold BUGS UCITS', 0.0),
             ('iShares Euro High Yield Corporate Bond UCITS',
              0.0017459068672136)])

In [ ]:
ef.max_sharpe()

OrderedDict([('Vanguard Eurozone Government Bond', 0.0),
             ('iShares Core Euro Corporate Bond UCITS', 0.0),
             ('db x-trackers II Barclays Global Aggregate Bond UC',
              0.1804126339799172),
             ('db x-trackers II iBoxx Liquid Corporate Financials',
              0.8195873660200828),
             ('db x-trackers USD Corporate Bond DR 2D Hedged', 0.0),
             ('iShares S&P Gold Producers UCITS', 0.0),
             ('ComStage NYSE Arca Gold BUGS UCITS', 0.0),
             ('iShares Euro High Yield Corporate Bond UCITS', 0.0)])

#**5. Visualizações**

In [ ]:
!pip install bt

     |████████████████████████████████| 2.7MB 5.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for bt: filename=bt-0.2.9-cp37-cp37m-linux_x86_64.whl size=765254 sha256=35443cb0093477b29d552836e0b26bf5641c76935b2051fc3ff6792960b2990b
  Stored in directory: /root/.cache/pip/wheels/18/2a/31/ead0e40a43116c3278ea50bdec622d43c9e51263b8a62ad71c
Successfully built bt


In [ ]:
import bt
 
import pandas as pd
import matplotlib
 
matplotlib.style.use('seaborn-darkgrid')
%matplotlib inline

In [ ]:
from pylab import mpl, plt
import scipy.stats as scs
import scipy.optimize as sco
import time
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [ ]:
rets = np.log(fechamentos/fechamentos.shift(1))
rets = rets.sort_values(by='Date', ascending=False)
rets = rets.iloc[:1260]
noa = len(rets)
rets = rets.sort_values(by='Date')
rets

,iShares Global Clean Energy UCITS,iShares MSCI China A UCITS USD,iShares STOXX Europe 600 UCITS,iShares S&P 500 USD Info Tech,db x-trackers MSCI China UCITS DR 1C,iShares MSCI Europe UCITS Dist,iShares S&P Gold Producers UCITS,iShares STOXX Europe 600 Oil & Gas UCITS,iShares STOXX Europe 600 Travel & Leisure UCITS,iShares Core MSCI World UCITS,db x-trackers CSI300 UCITS 1C,iShares MSCI World Momentum Factor UCITS,Vanguard S&P 500 UCITS USD Inc,iShares S&P 500 UCITS Dist,iShares STOXX Global Select Dividend 100 UCITS,iShares Core FTSE 100 UCITS,Vanguard FTSE Developed Europe,iShares Core MSCI Emerging Markets IMI UCITS,iShares S&P Commodity Producers Oil and Gas UCITS,db x-trackers MSCI World Info Tech DR 1C,ComStage NYSE Arca Gold BUGS UCITS,iShares NASDAQ-100 UCITS,SSgA SPDR S&P Euro Dividend Aristocrats,Lyxor UCITS New Energy D-EUR,UBS ETF MSCI World Socially Responsible UCITS USD,iShares TecDAX UCITS,iShares Global Water UCITS,iShares Euro High Yield Corporate Bond UCITS,Vanguard FTSE All-World UCITS USD Inc,iShares EURO Dividend UCITS,iShares AEX,iShares MSCI World UCITS Dist,iShares Core S&P 500 UCITS,iShares European Property Yield UCITS,iShares Listed Private Equity UCITS,iShares MSCI Turkey UCITS,iShares MSCI Emerging Markets UCITS Dist,iShares MSCI Japan UCITS Dist,SPDR Russell 2000 U.S. Small Cap UCITS,iShares EURO STOXX 50 UCITS Dist,iShares Developed Markets Property Yield UCITS,iShares MSCI World EUR Hedged UCITS,iShares MSCI Europe UCITS Acc,db x-trackers II iBoxx EUR High,db x-trackers USD Corporate Bond DR 2D Hedged,Deka STOXX Europe Strong Growth 20 UCITS,iShares Asia Pacific Dividend UCITS,iShares STOXX Europe Select Dividend 30 UCITS,Think AEX,Lyxor UCITS World Water D-EUR,Vanguard FTSE All-World High Div Yld,iShares Core Euro Corporate Bond UCITS,iShares S&P 500 EUR Hedged UCITS,iShares Global Infrastructure UCITS,db x-trackers II iBoxx Liquid Corporate Financials,db x-trackers II Barclays Global Aggregate Bond UC,iShares Bric 50 UCITS,Vanguard FTSE Emerging Markets UCITS USD Inc,iShares MSCI Korea UCITS Dist,SPDR MSCI World Technology,Think Global Equity,iShares MSCI Emerging Markets UCITS Acc,iShares China Large Cap UCITS,iShares MSCI Brazil UCITS Dist,Vanguard Eurozone Government Bond,Vanguard FTSE Developed Asia Pacif
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-05-18,-0.004784,-0.008826,0.007898,0.005386,0.001007,0.007641,-0.007404,0.004326,0.021163,0.003577,-0.010485,0.000000,0.001155,0.001654,0.001581,0.013915,0.007421,0.000000,-0.000672,0.006345,-0.005842,0.002618,-0.000973,0.004811,0.003734,0.001946,-0.003666,-0.000580,0.002746,0.002044,0.005019,0.002574,0.001997,-0.000528,0.004181,0.002742,0.000734,0.007715,-0.004212,0.005986,-0.008149,-0.003275,0.007059,-0.001162,-0.005559,0.023009,0.005303,0.006250,0.004768,-0.001496,0.002365,-0.001320,-0.004272,-0.000951,-0.000609,0.000000,-0.001640,0.001272,-0.010549,NaN,0.006223,0.000000,0.002243,-0.005672,-0.002388,-0.002754
2016-05-19,-0.025207,-0.003757,-0.010839,-0.007501,-0.015206,-0.011965,-0.051182,-0.026610,-0.015504,-0.008551,-0.001318,-0.004571,-0.004920,-0.006079,-0.011919,-0.010101,-0.011153,-0.016398,-0.028632,-0.007052,-0.053145,-0.007613,-0.009785,-0.012678,-0.009205,-0.009769,-0.009393,-0.001547,-0.008952,-0.007172,-0.011214,-0.007418,-0.006533,-0.017865,-0.014006,-0.024955,-0.017385,-0.011594,-0.010803,-0.012008,-0.012809,-0.011420,-0.010239,-0.002328,-0.002093,0.003303,-0.011971,-0.011278,-0.010932,-0.004202,-0.009972,-0.000777,-0.012275,-0.011965,-0.000406,0.002216,-0.019332,-0.019772,-0.001415,NaN,-0.008604,-0.018462,-0.013154,-0.045486,-0.000797,-0.011651
2016-05-20,0.024293,0.024342,0.014039,0.020725,0.016211,0.013391,-0.015766,0.015760,0.014649,0.013482,0.011803,0.012829,0.014115,0.014856,0.014680,0.011357,0.013737,0.016959,0.024591,0.020316,-0.021864,0.020861,0.010758,0.017465,0.015552,0.011064,0.016048,0.002610,0.013909,0.011756,0.014168,0.013186,0.015294,0.012034,0.015396,0.018545,0.017019,0.011594,0.0

In [ ]:
def port_ret(weights):
    return np.sum(rets.mean() * weights) * 252
def port_vol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))

In [ ]:
def min_func_sharpe(weights):
    return -port_ret(weights)/port_vol(weights)

In [ ]:
def ytd(ticker):
  prices = fechamentos[ticker].tail(252).reset_index()
  start = prices.iloc[0][1]
  end = prices.iloc[-1][1]
  return ((end-start)/start)*252

In [ ]:
df = pd.DataFrame(columns=['ticker', 'ytd'])
for tick in fechamentos.columns:
    chg = ytd(tick)
    row = [tick, chg]
    df.loc[len(df)] = row
filtered_df = df
#convert to a list so that we can plug them into our MPT function
new_list = filtered_df['ticker'].tolist()
new_list

['iShares Global Clean Energy UCITS',
 'iShares MSCI China A UCITS USD',
 'iShares STOXX Europe 600 UCITS',
 'iShares S&P 500 USD Info Tech',
 'db x-trackers MSCI China UCITS DR 1C',
 'iShares MSCI Europe UCITS Dist',
 'iShares S&P Gold Producers UCITS',
 'iShares STOXX Europe 600 Oil & Gas UCITS',
 'iShares STOXX Europe 600 Travel & Leisure UCITS',
 'iShares Core MSCI World UCITS',
 'db x-trackers CSI300 UCITS 1C',
 'iShares MSCI World Momentum Factor UCITS',
 'Vanguard S&P 500 UCITS USD Inc',
 'iShares S&P 500 UCITS Dist',
 'iShares STOXX Global Select Dividend 100 UCITS',
 'iShares Core FTSE 100 UCITS',
 'Vanguard FTSE Developed Europe',
 'iShares Core MSCI Emerging Markets IMI UCITS',
 'iShares S&P Commodity Producers Oil and Gas UCITS',
 'db x-trackers MSCI World Info Tech DR 1C',
 'ComStage NYSE Arca Gold BUGS UCITS',
 'iShares NASDAQ-100 UCITS',
 'SSgA SPDR S&P Euro Dividend Aristocrats',
 'Lyxor UCITS New Energy D-EUR',
 'UBS ETF MSCI World Socially Responsible UCITS USD',
 'iS

In [ ]:
merged_maiores_volumes.loc['Nome',selecao_final]

KeyError: ignored

In [ ]:
#retreive stock data
prices = fechamentos
noa = len(new_list)
#Calculate Daily Returns
rets = np.log(prices / prices.shift(1))
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1})
bnds = tuple((0,1) for x in range(noa))
eweights = np.array(noa * [1. / noa,])

In [ ]:
anual = pd.DataFrame(columns=['Nome', 'Retorno'])
 
for y in new_list:
  chg = ytd(y)
  row = [y,chg]
  anual.loc[len(anual)]=row
 
anual.sort_values(by='Retorno',ascending=False)

,Nome,Retorno
0,iShares Global Clean Energy UCITS,184.667368
8,iShares STOXX Europe 600 Travel & Leisure UCITS,167.823158
58,iShares MSCI Korea UCITS Dist,151.106291
38,SPDR Russell 2000 U.S. Small Cap UCITS,144.577177
18,iShares S&P Commodity Producers Oil and Gas UCITS,137.009709
...,...,...
44,db x-trackers USD Corporate Bond DR 2D Hedged,-13.174263
6,iShares S&P Gold Producers UCITS,-16.975000
20,ComStage NYSE Arca Gold BUGS UCITS,-19.109451
55,db x-trackers II Barclays Global Aggregate Bon...,-20.143931


In [ ]:
%%time
opts = sco.minimize(min_func_sharpe, eweights, method='SLSQP', bounds=bnds, constraints=cons)

CPU times: user 27.8 s, sys: 173 ms, total: 27.9 s
Wall time: 27.8 s


In [ ]:
print('The return is: {:.2f}'.format(port_ret(opts['x'].round(3))))
print('the volatility is: {:.2f}'.format(port_vol(opts['x'].round(3))))
print("The Sharpe Ratio is: {:.2f}".format(port_ret(opts['x']/port_vol(opts['x']))))

The return is: 0.10
the volatility is: 0.09
The Sharpe Ratio is: 1.16


In [ ]:
i = 0
j = 0
escolhidos = []
for x in opts['x']:
    if x > 0.01:
      escolhidos.append(new_list[i])
      print(new_list[i] + ": " + str(x.round(2)))
      j=j+1
    i = i + 1
escolhidos

db x-trackers II iBoxx Liquid Corporate Financials: 0.07
db x-trackers II Barclays Global Aggregate Bond UC: 0.05
SPDR MSCI World Technology: 0.4
Vanguard Eurozone Government Bond: 0.48


['db x-trackers II iBoxx Liquid Corporate Financials',
 'db x-trackers II Barclays Global Aggregate Bond UC',
 'SPDR MSCI World Technology',
 'Vanguard Eurozone Government Bond']

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1})
bnds = tuple((0,1) for x in range(noa))
eweights = np.array(noa * [1. / noa,])
 
#time
opts = sco.minimize(min_func_sharpe, eweights, method='SLSQP', bounds=bnds, constraints=cons)

In [ ]:
print('The return is: {:.2f}'.format(port_ret(opts['x'].round(3))))
print('the volatility is: {:.2f}'.format(port_vol(opts['x'].round(3))))
print("The Sharpe Ratio is: {:.2f}".format(port_ret(opts['x']/port_vol(opts['x']))))

The return is: 0.10
the volatility is: 0.10
The Sharpe Ratio is: 1.07
